# Introduction
This notebook is used to experiment with different prompts for the question generation model that uses the chatGPT API from open.ai.
First a helper function is created to call the API with the provided prompt. For this prompt different techniques are tried out and evaluated to find the best performing prompt template.

In [9]:
import os
from dotenv import load_dotenv
import openai
from src.datageneration.extractor import extract_text_without_image
from pypdfium2 import PdfDocument
import pandas as pd
from sklearn.model_selection import train_test_split
from src.evaluation.eval_main import Metrics
import nltk

nltk.download('wordnet')

load_dotenv()
openai.api_key = os.getenv("OPENAI-API-KEY2")

def chat_gpt(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

[nltk_data] Downloading package wordnet to /Users/I516258/nltk_data...


Now the evaluation data is loaded that is also used to evaluate the other question generation models that are used in the Ankinator project.

In [2]:
# # initially retrieve extracted text for each slide - only execute once
# slide_path = "../../../datasets/IT-Security_all_slides_no_duplicates.pdf"
# pdf = PdfDocument(slide_path)
# text = extract_text_without_image(pdf.raw)
# extracted_content = pd.DataFrame(columns=['Pagenumber', 'Page-Text', 'OCR-text'])
# for i in text:
#     extracted_content = extracted_content.append({'Pagenumber': i[0], 'Page-Text': i[1], 'OCR-text': i[2]}, ignore_index=True)
#
# # Define the file path and name
# file_path = "../../../datasets/extracted_text_content.csv"
#
# # Save the DataFrame to the specified folder
# extracted_content.to_csv(file_path, index=False)

100%|██████████| 596/596 [06:26<00:00,  1.54it/s]
/var/folders/96/q1m76wjs68d10zrlcykfyhth0000gn/T/ipykernel_32402/3497456139.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_content = extracted_content.append({'Pagenumber': i[0], 'Page-Text': i[1], 'OCR-text': i[2]}, ignore_index=True)
/var/folders/96/q1m76wjs68d10zrlcykfyhth0000gn/T/ipykernel_32402/3497456139.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_content = extracted_content.append({'Pagenumber': i[0], 'Page-Text': i[1], 'OCR-text': i[2]}, ignore_index=True)
/var/folders/96/q1m76wjs68d10zrlcykfyhth0000gn/T/ipykernel_32402/3497456139.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_content = extracted_content.append({'Pagenu

In [5]:
# reload extracted content from file
file_path = "../../../datasets/extracted_text_content.csv"
extracted_content = pd.read_csv(file_path)

In [6]:
file_path = '../../../datasets/Goldstandard.csv'

goldstandard = pd.read_csv(file_path, delimiter=';')

# Remove unnecessary columns
goldstandard.drop(['PDF-Name', 'Comment','Page Number'], axis=1, inplace=True)

# Join two DataFrames based on index
goldstandard = extracted_content.join(goldstandard, lsuffix='_left', rsuffix='_right')

# Delete records with value "No" and "no" in the "Marked for processing" column
goldstandard = goldstandard[(goldstandard['Marked for processing'] != 'No') & (goldstandard['Marked for processing'] != 'no')]

# Remove unnecessary columns
goldstandard.drop(['Marked for processing', 'Includes Image Data'], axis=1, inplace=True)

# Split the DataFrame into train, validation, and test sets
goldstandard_train_val, goldstandard_test = train_test_split(goldstandard, test_size=0.2, random_state=42)

print("Lenght of test set: ", len(goldstandard_test))
print(goldstandard_test)

Lenght of test set:  93
     Pagenumber                                          Page-Text  \
287         287  Phase 4: Post-Infection\r\n• Carries out attac...   
35           35  • Sig:\r\n– Given a message  ∈ ே and \r\nsecre...   
46           46  Access Control\r\n• Controls which authenticat...   
284         284  Phase 1: Development\r\n• Malware is developme...   
149         149  Phishing\r\n• Phishing = fishing for informati...   
..          ...                                                ...   
26           26  Key Agreement Protocol\r\nAlice Bob\r\nEve\r\n...   
449         449  HTTP\r\nWorkflow\r\n• Two parties:\r\n– Client...   
54           54  Access Matrix\r\nFormal Definition\r\n• At poi...   
217         217  Machine Learning\r\n• Extends content filterin...   
436         436  DNS Tunneling\r\n• Normal DNS requests only co...   

                                              OCR-text  \
287  te\nPhase 4: Post-Infection Be OP MANNHEIM\n\n...   
35   ol\nExample: R

In [11]:
# Reset the index of the DataFrame
goldstandard_test = goldstandard_test.reset_index(drop=True)
goldstandard_train_val = goldstandard_train_val.reset_index(drop=True)

# this stores now the possible input for the chatGPT model
content = goldstandard_test[["Page-Text", "OCR-text"]]

# this stores the reference
references = goldstandard_test[["Question"]]

                                             Question
0   How is the post-infection phase of the malware...
1             How does the RSA Signature Scheme work?
2                      How is Access Control defined?
3   How is the development phase of the Malware li...
4                                   What is Phishing?
..                                                ...
88  What is the workflow for a key agreement proto...
89                                How does HTTP work?
90   How is the formal definiton of an Access Matrix?
91  How is Machine Learning used to extend conten ...
92                             What is DNS Tunneling?

[93 rows x 1 columns]


# Prompt Engineering
Having prepared everything it is possible to start with prompt engineering. It is started with simple prompts and continued with more complex prompts.
## Zero-Shot
1. Simple prompt

In [18]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    ```{row['Page-Text']}```
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# Performance is evaluated
metrics = Metrics(save_to_file=False)
result = pd.DataFrame(
    metrics.evaluate(model_output=model_results, references=references[:3]),
    index=["ChatGPT"]
)
result

['List some examples of actions that a malware might carry out during the post-infection phase.', 'What is the process for generating a signature in the RSA Signature Scheme?', 'What is access control and on which levels can it exist?']
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling par

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/I516258/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

2. Provide more context on the task

In [74]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    When there are examples do not focus on their specifics but try to cover the overarching concept or idea.
    ```{row['Page-Text']}```
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

['What actions does a virus typically carry out during the post-infection phase?', 'What is the purpose of the "Sig" operation in the RSA Signature Scheme?', 'What does access control in IT security refer to and on what levels can it exist?']


3. Even more context

In [76]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    Focus on concepts, definitions and key-words.
    Take into account how exam questions are normally formulated and formulate the question accordingly.
    When there are examples do not focus on their specifics but try to cover the overarching concept or idea.
    ```{row['Page-Text']}```
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

['What are some examples of actions that can be carried out during the post-infection phase of a cyberattack?', 'What are the steps involved in the RSA signature scheme?', 'What does access control in IT-Security involve and what are the different levels at which it can exist?']


4. Comparison of copied text and OCR

In [79]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    You are a bot to support in the generation of flashcards from lecture slides.
    You are provided with two inputs. The first input delimited by triple backticks is the text that is copied from the slides.
    The second input delimited by triple quotation marks is retrieved with an OCR tool to extract all text from a slide.
    Follow the below process:
    1. Step: Compare the first input with the second input to retrieve the relevant information
    2. Step: Generate a question for this information in a flashcard style
    Only return the generated question.
    ```{row['Page-Text']}```
    \"\"\"{row['OCR-text']}\"\"\"
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

['What does "Phase 4: Post-Infection" entail?', 'What is the process of generating a signature in the RSA Signature Scheme?', 'What is the purpose of access control in a system?']


## Few-Shot
5. Few-Shot with simple question

In [1]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    ```{row['Page-Text']}```
    Follow a similar style for generating the question as in this two examples:
    1) Input: {goldstandard_train_val.loc[0, 'Page-Text']}, question: {goldstandard_train_val.loc[0, 'Question']}
    2) Input: {goldstandard_train_val.loc[1, 'Page-Text']}, question: {goldstandard_train_val.loc[1, 'Question']}
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(goldstandard_test.loc[1, 'Page-Text'])

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

NameError: name 'content' is not defined